In [10]:
%matplotlib inline
import imp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import seaborn as sns; sns.set()
import os
import pandas_profiling

pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.base import BaseEstimator, ClassifierMixin, RegressorMixin, ClusterMixin
from sklearn.model_selection import train_test_split

In [12]:
# Feature extractor
import pandas as pd
import os

class FeatureExtractor(object):
    def __init__(self):
        pass

    def fit(self, X_df, y_array):
        pass

    def transform(self, X_df):
        X_encoded = X_df
        path = os.path.dirname(__file__)
        # Feature engineering
        # Creating columns to distinguish between the two main airports for flights and the rest
        X_encoded['d_ManyFlights'] = 0
        X_encoded['a_ManyFlights'] = 0
        X_encoded['d_ManyFlights'][X_df['Departure'] == 'ORD'] = 1
        X_encoded['d_ManyFlights'][X_df['Departure'] == 'ATL'] = 1
        X_encoded['a_ManyFlights'][X_df['Arrival'] == 'ORD'] = 1
        X_encoded['a_ManyFlights'][X_df['Arrival'] == 'ATL'] = 1

        # Categorical encoding of departure and arrival airports
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Departure'], prefix='d'))
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Arrival'], prefix='a'))

        # Categorical encoding of the dates 
        X_encoded['DateOfDeparture'] = pd.to_datetime(X_encoded['DateOfDeparture'])
        X_encoded['year'] = X_encoded['DateOfDeparture'].dt.year
        X_encoded['month'] = X_encoded['DateOfDeparture'].dt.month
        X_encoded['day'] = X_encoded['DateOfDeparture'].dt.day
        X_encoded['weekday'] = X_encoded['DateOfDeparture'].dt.weekday
        X_encoded['week'] = X_encoded['DateOfDeparture'].dt.week
        X_encoded['n_days'] = X_encoded['DateOfDeparture'].apply(lambda date: (date - pd.to_datetime("1970-01-01")).days)

        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['year'], prefix='y'))
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['month'], prefix='m'))
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['day'], prefix='d'))
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['weekday'], prefix='wd'))
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['week'], prefix='w'))



        # Auxiliary dataset we will use to complete our data
        external_dataTest= pd.read_csv(os.path.join(path, 'external_data.csv'))
        external_data = external_dataTest['Date', 'AirPort', 'Max TemperatureC', 'Mean TemperatureC',
                                          'Min TemperatureC', 'Dew PointC', 'MeanDew PointC', 'Min DewpointC',
                                          'Max Humidity', 'Mean Humidity', 'Min Humidity',
                                          'Max Sea Level PressurehPa', 'Mean Sea Level PressurehPa',
                                          'Min Sea Level PressurehPa', 'Max VisibilityKm', 'Mean VisibilityKm',
                                          'Min VisibilitykM', 'Max Wind SpeedKm/h', 'Mean Wind SpeedKm/h',
                                          'Max Gust SpeedKm/h', 'Precipitationmm', 'CloudCover', 'Events',
                                          'WindDirDegrees', 'Rain', 'Thunderstorm', 'Fog', 'Snow', 'Hail',
                                          'Tornado', 'a_latitude_deg', 'a_longitude_deg', 'a_elevation_ft',
                                          '2018', '2017', '2016', '2015']

        # Now merging external data with out dataset 
        # Creating two tables, one for departures, the other for arrivals
        external_dataDeparture = external_data.rename(columns={'Date': 'DateOfDeparture', 'AirPort': 'Departure'})
        external_dataArrival = external_data.rename(columns={'Date': 'DateOfDeparture', 'AirPort': 'Arrival'})

        # Merging them with X_encoded   
        X_encoded = pd.merge(X_encoded, external_dataDeparture, how='left',left_on=['DateOfDeparture', 'Departure'],
                             right_on=['DateOfDeparture', 'Departure'],sort=False)
        X_encoded = pd.merge(X_encoded, external_dataArrival, how='left',left_on=['DateOfDeparture', 'Arrival'],
                             right_on=['DateOfDeparture', 'Arrival'],sort=False) 

        # Finally getting rid of departure, arrival, and date columns now that we do not need them to merge
        X_encoded = X_encoded.drop('Departure', axis=1)
        X_encoded = X_encoded.drop('Arrival', axis=1)
        X_encoded = X_encoded.drop('DateOfDeparture', axis=1)
        X_array = X_encoded.values

In [13]:
# Regressor 
from sklearn.base import BaseEstimator
from sklearn.linear_model import Ridge 
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoLars
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import RandomForestRegressor

class Regressor(BaseEstimator):
    def __init__(self):
        self.reg = RandomForestRegressor(n_estimators=20)
        self.reg2 = Lasso()
        self.reg3 = LassoLars()
        self.reg4 = Ridge()
        self.reg5 = LinearRegression()
        self.reg6 = BayesianRidge()
        self.metareg = RandomForestRegressor(n_estimators=20)

    def fit(self, X, y):
        self.reg.fit(X, y)
        self.reg2.fit(X, y)
        self.reg3.fit(X,y)
        self.reg4.fit(X,y)
        self.reg5.fit(X,y)
        X_combined = np.vstack([self.reg.predict(X), self.reg2.predict(X),self.reg3.predict(X),self.reg4.predict(X),self.reg5.predict(X)]).T
        self.metareg.fit(X_combined, y)


    def predict(self, X):
        pred1 = self.reg.predict(X)
        pred2 = self.reg2.predict(X)
        pred3 = self.reg3.predict(X)
        pred4 = self.reg4.predict(X)
        pred5 = self.reg5.predict(X)
        X_combined = np.vstack([pred1, pred2, pred3, pred4, pred5]).T
        return self.metareg.predict(X_combined)

In [16]:
features = X_encoded
X_columns = features.columns
X_array = features.values

X_train, X_test, y_train, y_test = train_test_split(X_array, y_array, test_size=0.2, random_state=0)

In [17]:
reg = Regressor()
reg.fit(X_train,y_train)

TypeError: float() argument must be a string or a number, not 'Timestamp'

### Getting the base dataset

In [6]:
problem = imp.load_source('', 'problem.py')
X_df, y_array = problem.get_train_data()

In [7]:
# Exploring the dataset
X_df.isna().sum()
X_df.describe()
X_df['Departure'].unique()
X_df['Arrival'].unique()
X_df.head()

,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd
0,2012-06-19,ORD,DFW,12.875000,9.812647
1,2012-09-10,LAS,DEN,14.285714,9.466734
2,2012-10-05,DEN,LAX,10.863636,9.035883
3,2011-10-09,ATL,ORD,11.480000,7.990202
4,2012-02-21,DEN,SFO,11.450000,9.517159


### Defining the feature extractor (WIP)

In [8]:
# Feature extractor test
X_encoded = X_df
# path = os.path.dirname(__file__)

# Feature engineering
# Creating columns to distinguish between the two main airports for flights and the rest
X_encoded['d_ManyFlights'] = 0
X_encoded['a_ManyFlights'] = 0
X_encoded['d_ManyFlights'][X_df['Departure'] == 'ORD'] = 1
X_encoded['d_ManyFlights'][X_df['Departure'] == 'ATL'] = 1
X_encoded['a_ManyFlights'][X_df['Arrival'] == 'ORD'] = 1
X_encoded['a_ManyFlights'][X_df['Arrival'] == 'ATL'] = 1

# Categorical encoding of departure and arrival airports
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Departure'], prefix='d'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Arrival'], prefix='a'))

# Categorical encoding of the dates 
X_encoded['DateOfDeparture'] = pd.to_datetime(X_encoded['DateOfDeparture'])
X_encoded['year'] = X_encoded['DateOfDeparture'].dt.year
X_encoded['month'] = X_encoded['DateOfDeparture'].dt.month
X_encoded['day'] = X_encoded['DateOfDeparture'].dt.day
X_encoded['weekday'] = X_encoded['DateOfDeparture'].dt.weekday
X_encoded['week'] = X_encoded['DateOfDeparture'].dt.week
X_encoded['n_days'] = X_encoded['DateOfDeparture'].apply(lambda date: (date - pd.to_datetime("1970-01-01")).days)

X_encoded = X_encoded.join(pd.get_dummies(X_encoded['year'], prefix='y'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['month'], prefix='m'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['day'], prefix='d'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['weekday'], prefix='wd'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['week'], prefix='w'))



# Auxiliary dataset we will use to complete our data

# external_dataTest= pd.read_csv(os.path.join(path, 'external_dataTest.csv'))
#external_data = external_dataTest['Date', 'AirPort', 'Max TemperatureC', 'Mean TemperatureC',
#       'Min TemperatureC', 'Dew PointC', 'MeanDew PointC', 'Min DewpointC',
#       'Max Humidity', 'Mean Humidity', 'Min Humidity',
#       'Max Sea Level PressurehPa', 'Mean Sea Level PressurehPa',
#       'Min Sea Level PressurehPa', 'Max VisibilityKm', 'Mean VisibilityKm',
#       'Min VisibilitykM', 'Max Wind SpeedKm/h', 'Mean Wind SpeedKm/h',
#       'Max Gust SpeedKm/h', 'Precipitationmm', 'CloudCover', 'Events',
#       'WindDirDegrees', 'Rain', 'Thunderstorm', 'Fog', 'Snow', 'Hail',
#       'Tornado', 'a_latitude_deg', 'a_longitude_deg', 'a_elevation_ft',
#       '2018', '2017', '2016', '2015']

# Now merging external data with out dataset 
# Creating two tables, one for departures, the other for arrivals
external_dataDeparture = external_data.rename(columns={'Date': 'DateOfDeparture', 'AirPort': 'Departure'})
external_dataArrival = external_data.rename(columns={'Date': 'DateOfDeparture', 'AirPort': 'Arrival'})

# Merging them with X_encoded   
X_encoded = pd.merge(X_encoded, external_dataDeparture, how='left',left_on=['DateOfDeparture', 'Departure'],
                             right_on=['DateOfDeparture', 'Departure'],sort=False)
X_encoded = pd.merge(X_encoded, external_dataArrival, how='left',left_on=['DateOfDeparture', 'Arrival'],
                             right_on=['DateOfDeparture', 'Arrival'],sort=False) 

# Finally getting rid of departure, arrival, and date columns now that we do not need them to merge
X_encoded = X_encoded.drop('Departure', axis=1)
X_encoded = X_encoded.drop('Arrival', axis=1)
X_encoded = X_encoded.drop('DateOfDeparture', axis=1)
X_array = X_encoded.values

NameError: name 'external_data' is not defined

In [ ]:
X_encoded.head()

### Getting auxiliary datasets

#### External data

In [20]:
data_weather = pd.read_csv("submissions/starting_kit/external_data.csv", sep = ",")
data_weather.head()

,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees
0,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,NaN,129
1,2011-09-02,ATL,36,29,22,17,15,14,61,46,30,1019,1016,1014,16,16,16,24,7,34.0,0.00,2,NaN,185
2,2011-09-03,ATL,35,29,23,17,16,14,64,47,30,1015,1013,1011,16,16,16,19,7,26.0,0.00,4,NaN,147
3,2011-09-04,ATL,27,24,22,22,19,16,93,72,51,1014,1012,1011,16,14,4,21,9,26.0,6.10,6,Rain,139
4,2011-09-05,ATL,26,24,22,23,22,20,94,91,87,1010,1005,999,16,13,3,32,16,45.0,16.00,8,Rain-Thunderstorm,149


In [21]:
data_weather.isna().sum()

Date                             0
AirPort                          0
Max TemperatureC                 0
Mean TemperatureC                0
Min TemperatureC                 0
Dew PointC                       0
MeanDew PointC                   0
Min DewpointC                    0
Max Humidity                     0
Mean Humidity                    0
Min Humidity                     0
Max Sea Level PressurehPa        0
Mean Sea Level PressurehPa       0
Min Sea Level PressurehPa        0
Max VisibilityKm                 0
Mean VisibilityKm                0
Min VisibilitykM                 0
Max Wind SpeedKm/h               0
Mean Wind SpeedKm/h              0
Max Gust SpeedKm/h             290
Precipitationmm                  0
CloudCover                       0
Events                        6745
WindDirDegrees                   0
dtype: int64

In [22]:
data_weather.profile_report()

In [23]:
# Getting rid of the T values and replacing them by a very small value
data_weather[data_weather["Precipitationmm"]=="T"] = 0.10

In [25]:
# Affecting rain to the events we do not know
data_weather["Events"][data_weather["Events"].isna()] = "Rain"

# Categorical encoding of the Events column
new = data_weather["Events"].str.split('-',expand=True)
events_list = ["Rain", "Thunderstorm", "Fog", "Snow", "Hail", "Tornado"]
for event in events_list:
    event_bool = [new == event]
    event_bool = event_bool[0]
    data_weather[event] = event_bool.sum(axis = 1)
data_weather = data_weather.drop(columns = "Events")
data_weather.head()

# Changing the date column to the right format
data_weather['Date'] = pd.to_datetime(data_weather['Date'])

# Dropping irrelevant, highly correlated columns
data_weather = data_weather.drop(columns = ["Max_Gust_SpeedKm/h","Mean_Humidity","Mean_Sea_Level_PressurehPa",
                                            "Mean_TemperatureC", "Dew_PointC", "Min_DewpointC", 
                                            "Min_Humidity","Min_Sea_Level_PressurehPa", "Min_TemperatureC"])
# Finally getting a look at our dataset
data_weather.head()

,Date,AirPort,Max_TemperatureC,MeanDew_PointC,Max_Humidity,Max_Sea_Level_PressurehPa,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm/h,Mean_Wind_SpeedKm/h,Precipitationmm,CloudCover,WindDirDegrees,Rain,Thunderstorm,Fog,Snow,Hail,Tornado
0,2011-09-01,ATL,35.0,18.0,79.0,1022.0,16.0,16.0,11.0,19.0,6.0,0.00,3.0,129.0,1,0,0,0,0,0
1,2011-09-02,ATL,36.0,15.0,61.0,1019.0,16.0,16.0,16.0,24.0,7.0,0.00,2.0,185.0,1,0,0,0,0,0
2,2011-09-03,ATL,35.0,16.0,64.0,1015.0,16.0,16.0,16.0,19.0,7.0,0.00,4.0,147.0,1,0,0,0,0,0
3,2011-09-04,ATL,27.0,19.0,93.0,1014.0,16.0,14.0,4.0,21.0,9.0,6.10,6.0,139.0,1,0,0,0,0,0
4,2011-09-05,ATL,26.0,22.0,94.0,1010.0,16.0,13.0,3.0,32.0,16.0,16.00,8.0,149.0,1,1,0,0,0,0


In [21]:
#data_weatherArrival = weather_data.rename(columns = {"Airport":"Arrival"})
#data_weatherDeparture = weather_data.rename(columns = {"Airport":"Arrival"})

In [22]:
data_weather.columns

Index(['Date', 'AirPort', 'Max_TemperatureC', 'Mean_TemperatureC',
       'Min_TemperatureC', 'Dew_PointC', 'MeanDew_PointC', 'Min_DewpointC',
       'Max_Humidity', 'Mean_Humidity', 'Min_Humidity',
       'Max_Sea_Level_PressurehPa', 'Mean_Sea_Level_PressurehPa',
       'Min_Sea_Level_PressurehPa', 'Max_VisibilityKm', 'Mean_VisibilityKm',
       'Min_VisibilitykM', 'Max_Wind_SpeedKm/h', 'Mean_Wind_SpeedKm/h',
       'Max_Gust_SpeedKm/h', 'Precipitationmm', 'CloudCover', 'WindDirDegrees',
       'Rain', 'Thunderstorm', 'Fog', 'Snow', 'Hail', 'Tornado'],
      dtype='object')

#### Airport data

In [26]:
data_geog = pd.read_csv("https://ourairports.com/data/airports.csv", sep = ',',verbose=False)
# Selecting the countries and airports we are interested in  
data_geog = data_geog[data_geog["iso_country"] == "US"]
data_geog = data_geog[data_geog["type"]=="large_airport"]
data_geog.head()

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords
26255,16091,KABQ,large_airport,Albuquerque International Sunport,35.040199,-106.609001,5355.0,NaN,US,US-NM,Albuquerque,yes,KABQ,ABQ,ABQ,http://www.abqsunport.com/,https://en.wikipedia.org/wiki/Albuquerque_Inte...,NaN
26274,3364,KADW,large_airport,Joint Base Andrews,38.810799,-76.866997,280.0,NaN,US,US-MD,Camp Springs,no,KADW,ADW,ADW,http://www.jba.af.mil/,https://en.wikipedia.org/wiki/Joint_Base_Andrews,Andrews Air Force Base
26287,3366,KAFW,large_airport,Fort Worth Alliance Airport,32.987598,-97.318802,722.0,NaN,US,US-TX,Fort Worth,no,KAFW,AFW,AFW,http://www.allianceairport.com/,https://en.wikipedia.org/wiki/Fort_Worth_Allia...,NaN
26291,3368,KAGS,large_airport,Augusta Regional At Bush Field,33.369900,-81.964500,144.0,NaN,US,US-GA,Augusta,yes,KAGS,AGS,AGS,NaN,https://en.wikipedia.org/wiki/Augusta_Regional...,NaN
26324,3377,KAMA,large_airport,Rick Husband Amarillo International Airport,35.219398,-101.706001,3607.0,NaN,US,US-TX,Amarillo,yes,KAMA,AMA,AMA,http://airport.amarillo.gov/,https://en.wikipedia.org/wiki/Rick_Husband_Ama...,NaN


In [27]:
# Creating a dictionary to map a IATA code to the name of the airport and select only those we want
my_dict1 = {'ORD':"Chicago O'Hare International Airport", 'LAS':'McCarran International Airport', 
 'DEN':'Denver International Airport', 'ATL':'Hartsfield Jackson Atlanta International Airport', 
 'SFO':'San Francisco International Airport', 'EWR':'Newark Liberty International Airport',
 'IAH':'George Bush Intercontinental Houston Airport', 'LAX':'Los Angeles International Airport', 
 'DFW':'Dallas Fort Worth International Airport', 'SEA': 'Seattle Tacoma International Airport', 
 'JFK':'John F Kennedy International Airport', 'PHL':'Philadelphia International Airport', 
 'MIA':'Miami International Airport', 'DTW': 'Detroit Metropolitan Wayne County Airport',
 'BOS':'General Edward Lawrence Logan International Airport', 'MSP': 'Minneapolis-St Paul International/Wold-Chamberlain Airport', 
 'CLT':'Charlotte Douglas International Airport', 'MCO':'Orlando International Airport', 
 'PHX':'Phoenix Sky Harbor International Airport', 'LGA':'La Guardia Airport'}
my_dict2 = {y:x for x,y in my_dict1.items()}

In [28]:
# Selecting only the airports that are in our initial dataset
data_geog['IATA'] = data_geog['name'].map(my_dict2)
data_geog = data_geog[data_geog['IATA'].isna() == False]

In [29]:
data_geog = data_geog.drop(columns = ["id", "ident", "type", "continent", "keywords","gps_code",
                                      "local_code","home_link","wikipedia_link", "iata_code", 
                                      "name", "iso_region", "municipality", "iso_country", "scheduled_service"])
data_geog = data_geog.rename(columns={"IATA":"AirPort"})
data_geog.head()

,latitude_deg,longitude_deg,elevation_ft,AirPort
26376,33.636700,-84.428101,1026.0,ATL
26502,42.364300,-71.005203,20.0,BOS
26663,35.214001,-80.943100,748.0,CLT
26805,39.861698,-104.672997,5431.0,DEN
26810,32.896801,-97.038002,607.0,DFW


In [27]:
# Departure 
# d_data_geog = data_geog
# d_data_geog.columns = ['d_latitude_deg', 'd_longitude_deg', 'd_elevation_ft', 'Departure']
# d_data_geog.columns

In [28]:
# Arrival
# a_data_geog = data_geog
# a_data_geog.columns = ['a_latitude_deg', 'a_longitude_deg', 'a_elevation_ft', 'Arrival']
# a_data_geog.columns

In [30]:
external_data = pd.merge(data_weather, data_geog, how='left',left_on=['AirPort'], right_on=['AirPort'],sort=False)
external_data.head()

,Date,AirPort,Max_TemperatureC,MeanDew_PointC,Max_Humidity,Max_Sea_Level_PressurehPa,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm/h,Mean_Wind_SpeedKm/h,Precipitationmm,CloudCover,WindDirDegrees,Rain,Thunderstorm,Fog,Snow,Hail,Tornado,latitude_deg,longitude_deg,elevation_ft
0,2011-09-01,ATL,35.0,18.0,79.0,1022.0,16.0,16.0,11.0,19.0,6.0,0.00,3.0,129.0,1,0,0,0,0,0,33.6367,-84.428101,1026.0
1,2011-09-02,ATL,36.0,15.0,61.0,1019.0,16.0,16.0,16.0,24.0,7.0,0.00,2.0,185.0,1,0,0,0,0,0,33.6367,-84.428101,1026.0
2,2011-09-03,ATL,35.0,16.0,64.0,1015.0,16.0,16.0,16.0,19.0,7.0,0.00,4.0,147.0,1,0,0,0,0,0,33.6367,-84.428101,1026.0
3,2011-09-04,ATL,27.0,19.0,93.0,1014.0,16.0,14.0,4.0,21.0,9.0,6.10,6.0,139.0,1,0,0,0,0,0,33.6367,-84.428101,1026.0
4,2011-09-05,ATL,26.0,22.0,94.0,1010.0,16.0,13.0,3.0,32.0,16.0,16.00,8.0,149.0,1,1,0,0,0,0,33.6367,-84.428101,1026.0


In [ ]:
#X_encoded = pd.merge(X_df, airport_arrival, how='left',left_on=['Arrival'], right_on=['Arrival'],sort=False)
#X_encoded = pd.merge(X_encoded, airport_departure, how='left',left_on=['Departure'], right_on=['Departure'],
                     #sort=False)
#X_encoded.head()

#### Passenger traffic

In [31]:
data_traffic = pd.read_csv("data/passenger_traff.csv", sep = ",")

In [32]:
data_traffic = data_traffic.drop(columns = ["Airports (large hubs)", "Major city served", "State"])
data_traffic.head()

,Rank_2018,IATA,2018,2017,2016,2015
0,1,ATL,"51,866,464","50,251,964","50,501,858","49,340,732"
1,2,LAX,"42,626,783","41,232,432","39,636,042","36,351,226"
2,3,ORD,"39,874,879","38,593,028","37,589,899","36,305,668"
3,4,DFW,"32,800,721","31,816,933","31,283,579","31,589,832"
4,5,DEN,"31,363,573","29,809,097","28,267,394","26,280,043"


In [33]:
# Cleaning the dataset to use the numbers 
columns = ["Rank_2018", "2018","2017","2016","2015"]
for column in columns: 
    data_traffic.loc[:,column] = data_traffic.loc[:, column].str.replace('\xa0','')
    data_traffic.loc[:,column] = data_traffic.loc[:, column].str.replace(',','')
    data_traffic.loc[:,column] = list(map(int,data_traffic.loc[:,column]))
    
data_traffic.loc[:,"IATA"] = data_traffic.loc[:,"IATA"].str.replace('\xa0','')

# Dropping the rank which we won't use 
data_traffic = data_traffic.drop(columns = ["Rank_2018"])

In [37]:
# Defining arrival 
#X_PassengerArrival = X_PassengerTraffic.rename(columns={'IATA': 'Arrival'})
#X_PassengerArrival.columns = ['Arrival', 'a_2018', 'a_2017', 'a_2016', 'a_2015']
#X_PassengerArrival.head()

# Defining departure
#X_PassengerDeparture = X_PassengerTraffic.rename(columns={'IATA': 'Departure'})
#X_PassengerDeparture.columns = ['Departure', 'd_2018', 'd_2017', 'd_2016', 'd_2015']
#X_PassengerDeparture.head()

In [38]:
# X_PassengerTraffic_arrival = X_PassengerTraffic.join(pd.get_dummies(X_PassengerTraffic["Arrival"], prefix='a'))
# X_PassengerTraffic_departure = X_PassengerTraffic.join(pd.get_dummies(X_PassengerTraffic["Departure"], prefix='d'))

In [39]:
#X_encoded = pd.merge(X_df, X_PassengerArrival, how='left',left_on=['Arrival'], right_on=['Arrival'],sort=False)
#X_encoded = pd.merge(X_encoded, X_PassengerDeparture, how='left',left_on=['Departure'], right_on=['Departure'],
#                    sort=False)
#X_encoded.head()

In [34]:
data_traffic = data_traffic.rename(columns={'IATA': 'AirPort'})

In [35]:
external_data = pd.merge(external_data, data_traffic, how='left',left_on=['AirPort'], right_on=['AirPort'],sort=False)

In [36]:
external_data.head()

,Date,AirPort,Max_TemperatureC,MeanDew_PointC,Max_Humidity,Max_Sea_Level_PressurehPa,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm/h,Mean_Wind_SpeedKm/h,Precipitationmm,CloudCover,WindDirDegrees,Rain,Thunderstorm,Fog,Snow,Hail,Tornado,latitude_deg,longitude_deg,elevation_ft,2018,2017,2016,2015
0,2011-09-01,ATL,35.0,18.0,79.0,1022.0,16.0,16.0,11.0,19.0,6.0,0.00,3.0,129.0,1,0,0,0,0,0,33.6367,-84.428101,1026.0,51866464.0,50251964.0,50501858.0,49340732.0
1,2011-09-02,ATL,36.0,15.0,61.0,1019.0,16.0,16.0,16.0,24.0,7.0,0.00,2.0,185.0,1,0,0,0,0,0,33.6367,-84.428101,1026.0,51866464.0,50251964.0,50501858.0,49340732.0
2,2011-09-03,ATL,35.0,16.0,64.0,1015.0,16.0,16.0,16.0,19.0,7.0,0.00,4.0,147.0,1,0,0,0,0,0,33.6367,-84.428101,1026.0,51866464.0,50251964.0,50501858.0,49340732.0
3,2011-09-04,ATL,27.0,19.0,93.0,1014.0,16.0,14.0,4.0,21.0,9.0,6.10,6.0,139.0,1,0,0,0,0,0,33.6367,-84.428101,1026.0,51866464.0,50251964.0,50501858.0,49340732.0
4,2011-09-05,ATL,26.0,22.0,94.0,1010.0,16.0,13.0,3.0,32.0,16.0,16.00,8.0,149.0,1,1,0,0,0,0,33.6367,-84.428101,1026.0,51866464.0,50251964.0,50501858.0,49340732.0


In [37]:
external_data_new = external_data.to_csv('external_data.csv', index = None, header=True) 
#Don't forget to add '.csv' at the end of the path
print(external_data_new)

None


### Moving on to predictions

In [45]:
features = X_encoded.drop("Events_x",axis=1)
X_columns = features.columns
X_array = features.values

X_train, X_test, y_train, y_test = train_test_split(X_array, y_array, test_size=0.2, random_state=0)

NameError: name 'X_encoded' is not defined

In [12]:
regressors = [Ridge(),Lasso(),RandomForestRegressor(),LassoLars()]

In [13]:
class Regressor(BaseEstimator) :
    def __init__(self):
        self.reg = []
        regressors = [Ridge(),Lasso(),RandomForestRegressor(),LassoLars()]
        for regressor in regressors : 
            self.reg.append(regressor)
        self.metareg = RandomForestRegressor(n_estimators=20)  
    
    def fit(self, X, y):
        (n,p) = X.shape
        X_combined = np.empty((1,n))
        for i in np.arange(len(regressors)):
            self.reg[i].fit(X,y)
            X_combined = np.vstack([X_combined, self.reg[i].predict(X)])
        X_combined = X_combined.T
        self.metareg.fit(X_combined, y)
    
    def predict(self, X):
        for i in np.arange(len(regressors)):
            X_combined = np.vstack([X_combined, self.reg[i].predict(X)])
        X_combined = X_combined.T
        return self.metareg.predict(X_combined)

In [14]:
reg = Regressor()
reg.fit(X_train,y_train)

In [61]:
X_train.shape

(7121, 153)

In [18]:
# Getting the scores on the train data 
scores = cross_val_score(reg, X_train, y_train, cv=5, scoring='neg_mean_squared_error',n_jobs=3)
print("RMSE: {:.4f} +/- {:.4f}".format(
    np.mean(np.sqrt(-scores)), np.std(np.sqrt(-scores))))

In [19]:
# Getting the scores on the test data
scores = cross_val_score(reg, X_test, y_test, cv=5, scoring='neg_mean_squared_error')
print("RMSE: {:.4f} +/- {:.4f}".format(np.mean(np.sqrt(-scores)), np.std(np.sqrt(-scores))))

In [44]:
# Regressor 
from sklearn.base import BaseEstimator
from sklearn.linear_model import Ridge 
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoLars
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import RandomForestRegressor

class Regressor(BaseEstimator):
    def __init__(self):
        self.reg = RandomForestRegressor(n_estimators=20)
        self.reg2 = Lasso()
        self.reg3 = LassoLars()
        self.reg4 = Ridge()
        self.reg5 = LinearRegression()
        self.reg6 = BayesianRidge()
        self.metareg = RandomForestRegressor(n_estimators=20)

    def fit(self, X, y):
        self.reg.fit(X, y)
        self.reg2.fit(X, y)
        self.reg3.fit(X,y)
        self.reg4.fit(X,y)
        self.reg5.fit(X,y)
        X_combined = np.vstack([self.reg.predict(X), self.reg2.predict(X),self.reg3.predict(X),self.reg4.predict(X),self.reg5.predict(X)]).T
        self.metareg.fit(X_combined, y)


    def predict(self, X):
        pred1 = self.reg.predict(X)
        pred2 = self.reg2.predict(X)
        pred3 = self.reg3.predict(X)
        pred4 = self.reg4.predict(X)
        pred5 = self.reg5.predict(X)
        X_combined = np.vstack([pred1, pred2, pred3, pred4, pred5]).T
        return self.metareg.predict(X_combined)

In [100]:
reg = Regressor()
reg.fit(X_train,y_train)

ValueError: could not convert string to float: 'Rain'

In [36]:
# Getting the scores on the train data
scores = cross_val_score(reg, X_train, y_train, cv=5, scoring='neg_mean_squared_error',n_jobs=3)
print("RMSE: {:.4f} +/- {:.4f}".format(
    np.mean(np.sqrt(-scores)), np.std(np.sqrt(-scores))))

RMSE: 0.4961 +/- 0.0116


In [37]:
# Getting the scores on the test data
scores = cross_val_score(reg, X_test, y_test, cv=5, scoring='neg_mean_squared_error')
print("RMSE: {:.4f} +/- {:.4f}".format(
    np.mean(np.sqrt(-scores)), np.std(np.sqrt(-scores))))

RMSE: 0.6153 +/- 0.0324


In [106]:
# Insert baseline regressor here
from sklearn.ensemble import RandomForestRegressor
from sklearn.base import BaseEstimator
class Regressor2(BaseEstimator):
    def __init__(self):
        self.reg = RandomForestRegressor(
            n_estimators=20, max_depth=50, max_features=10)

    def fit(self, X, y):
        self.reg.fit(X, y)

    def predict(self, X):
        return self.reg.predict(X)

In [107]:
reg2 = Regressor2()
reg2.fit(X_train,y_train)

ValueError: could not convert string to float: 'T'

In [24]:
# Getting the scores on the train data
scores = cross_val_score(reg2, X_train, y_train, cv=5, scoring='neg_mean_squared_error',n_jobs=3)
print("RMSE: {:.4f} +/- {:.4f}".format(
    np.mean(np.sqrt(-scores)), np.std(np.sqrt(-scores))))

RMSE: 0.6039 +/- 0.0158


In [25]:
# Getting the scores on the test data
scores = cross_val_score(reg2, X_test, y_test, cv=5, scoring='neg_mean_squared_error')
print("RMSE: {:.4f} +/- {:.4f}".format(
    np.mean(np.sqrt(-scores)), np.std(np.sqrt(-scores))))

RMSE: 0.6962 +/- 0.0212


In [69]:
# Insert test regressor here
from sklearn.ensemble import RandomForestRegressor
from sklearn.base import BaseEstimator
from sklearn.linear_model import LogisticRegression
class Regressor2(BaseEstimator):
    def __init__(self):
        self.reg = LinearRegression()

    def fit(self, X, y):
        self.reg.fit(X, y)

    def predict(self, X):
        return self.reg.predict(X)

In [70]:
reg2 = Regressor2()
reg2.fit(X_train,y_train)

In [71]:
# Getting the scores on the train data
scores = cross_val_score(reg2, X_train, y_train, cv=5, scoring='neg_mean_squared_error',n_jobs=3)
print("RMSE: {:.4f} +/- {:.4f}".format(
    np.mean(np.sqrt(-scores)), np.std(np.sqrt(-scores))))

RMSE: 0.6298 +/- 0.0059


In [73]:
# Getting the scores on the test data
scores = cross_val_score(reg2, X_test, y_test, cv=5, scoring='neg_mean_squared_error')
print("RMSE: {:.4f} +/- {:.4f}".format(
    np.mean(np.sqrt(-scores)), np.std(np.sqrt(-scores))))

RMSE: 217824.4508 +/- 435647.5722


In [10]:
# Feature extractor 
import pandas as pd
import os


class FeatureExtractor(object):
    def __init__(self):
        pass

    def fit(self, X_df, y_array):
        pass

    def transform(self, X_df):
        X_encoded = X_df
        path = os.path.dirname(__file__)
        data_weather = pd.read_csv(os.path.join(path, 'external_data.csv'))
        X_weather = data_weather[['Date', 'AirPort', 'Max TemperatureC']]
        X_weatherArrival = X_weather.rename(columns={'Date': 'DateOfDeparture', 'AirPort': 'Arrival'})
        X_encoded = pd.merge(X_encoded, X_weatherArrival, how='left',left_on=['DateOfDeparture', 'Arrival'],
                             right_on=['DateOfDeparture', 'Arrival'],sort=False)
        
        X_weatherDeparture = X_weather.rename(columns={'Date': 'DateOfDeparture', 'AirPort': 'Departure'})
        X_encoded = pd.merge(X_encoded, X_weatherDeparture, how='left',left_on=['DateOfDeparture', 'Departure'],
                             right_on=['DateOfDeparture', 'Departure'],sort=False)

        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Departure'], prefix='d'))
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Arrival'], prefix='a'))
        X_encoded = X_encoded.drop('Departure', axis=1)
        X_encoded = X_encoded.drop('Arrival', axis=1)

        X_encoded = X_encoded.drop('DateOfDeparture', axis=1)
        X_array = X_encoded.values
        return X_array

In [ ]:
# Feature extractor
import pandas as pd
import os

class FeatureExtractor(object):
    def __init__(self):
        pass

    def fit(self, X_df, y_array):
        pass

    def transform(self, X_df):
        X_encoded = X_df
        path = os.path.dirname(__file__)
        # Feature engineering
        # Creating columns to distinguish between the two main airports for flights and the rest
        X_encoded['d_ManyFlights'] = 0
        X_encoded['a_ManyFlights'] = 0
        X_encoded['d_ManyFlights'][X_df['Departure'] == 'ORD'] = 1
        X_encoded['d_ManyFlights'][X_df['Departure'] == 'ATL'] = 1
        X_encoded['a_ManyFlights'][X_df['Arrival'] == 'ORD'] = 1
        X_encoded['a_ManyFlights'][X_df['Arrival'] == 'ATL'] = 1

        # Categorical encoding of departure and arrival airports
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Departure'], prefix='d'))
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Arrival'], prefix='a'))

        # Categorical encoding of the dates 
        X_encoded['DateOfDeparture'] = pd.to_datetime(X_encoded['DateOfDeparture'])
        X_encoded['year'] = X_encoded['DateOfDeparture'].dt.year
        X_encoded['month'] = X_encoded['DateOfDeparture'].dt.month
        X_encoded['day'] = X_encoded['DateOfDeparture'].dt.day
        X_encoded['weekday'] = X_encoded['DateOfDeparture'].dt.weekday
        X_encoded['week'] = X_encoded['DateOfDeparture'].dt.week
        X_encoded['n_days'] = X_encoded['DateOfDeparture'].apply(lambda date: (date - pd.to_datetime("1970-01-01")).days)

        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['year'], prefix='y'))
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['month'], prefix='m'))
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['day'], prefix='d'))
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['weekday'], prefix='wd'))
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['week'], prefix='w'))



        # Auxiliary dataset we will use to complete our data
        external_dataTest= pd.read_csv(os.path.join(path, 'external_dataTest.csv'))
        external_data = external_dataTest['Date', 'AirPort', 'Max TemperatureC', 'Mean TemperatureC',
                                          'Min TemperatureC', 'Dew PointC', 'MeanDew PointC', 'Min DewpointC',
                                          'Max Humidity', 'Mean Humidity', 'Min Humidity',
                                          'Max Sea Level PressurehPa', 'Mean Sea Level PressurehPa',
                                          'Min Sea Level PressurehPa', 'Max VisibilityKm', 'Mean VisibilityKm',
                                          'Min VisibilitykM', 'Max Wind SpeedKm/h', 'Mean Wind SpeedKm/h',
                                          'Max Gust SpeedKm/h', 'Precipitationmm', 'CloudCover', 'Events',
                                          'WindDirDegrees', 'Rain', 'Thunderstorm', 'Fog', 'Snow', 'Hail',
                                          'Tornado', 'a_latitude_deg', 'a_longitude_deg', 'a_elevation_ft',
                                          '2018', '2017', '2016', '2015']

        # Now merging external data with out dataset 
        # Creating two tables, one for departures, the other for arrivals
        external_dataDeparture = external_data.rename(columns={'Date': 'DateOfDeparture', 'AirPort': 'Departure'})
        external_dataArrival = external_data.rename(columns={'Date': 'DateOfDeparture', 'AirPort': 'Arrival'})

        # Merging them with X_encoded   
        X_encoded = pd.merge(X_encoded, external_dataDeparture, how='left',left_on=['DateOfDeparture', 'Departure'],
                             right_on=['DateOfDeparture', 'Departure'],sort=False)
        X_encoded = pd.merge(X_encoded, external_dataArrival, how='left',left_on=['DateOfDeparture', 'Arrival'],
                             right_on=['DateOfDeparture', 'Arrival'],sort=False) 

        # Finally getting rid of departure, arrival, and date columns now that we do not need them to merge
        X_encoded = X_encoded.drop('Departure', axis=1)
        X_encoded = X_encoded.drop('Arrival', axis=1)
        X_encoded = X_encoded.drop('DateOfDeparture', axis=1)
        X_array = X_encoded.values

In [ ]:
import os
import glob

os.chdir("/mydir")
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])

#export to csv
combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

